# Integrateur pour l'équation de la chaleur dans différents systèmes

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import matplotlib

import pandas as pd
import ipywidgets as ipw

from scipy.interpolate import interp1d
from scipy.linalg import expm
from bokeh.palettes import RdYlBu11
from bokeh.transform import linear_cmap
from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, ColorBar, BasicTickFormatter, BasicTicker, FixedTicker, FuncTickFormatter
from bokeh.client import push_session

from ipywidgets import interact
from ipywidgets import IntSlider, interact, FloatSlider
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
matplotlib.rcParams['figure.figsize'] = (16,16)
matplotlib.rcParams.update({'font.size': 18})

output_notebook()

Loading BokehJS ...

## 1) Thermalisation de deux blocs en contact thermique

- Deux blocs de métal sont mis en contact l'un avec l'autre, l'un a température $T_1$ et l'autre à température $T_2 \neq T_1$. On réalise une coupe des deux blocs, modélisée par une grille 2D NxN, où N est entier. Les deux blocs sont constitués du même nombre de moles de métal. Le système total formé des deux blocs est supposé isolé. 

- Le code suivant vise à modéliser l'évolution temporelle du profil de température à l'interface entre les deux blocs, due à un transfert de chaleur d'un bloc à l'autre. 

- En output du code, un plot de la vue en coupe des deux au temps final, idem pour le profil de température.

### Utilisation:

 - Commencer par mettre les deux curseurs au minimum pour visualiser la configuration initiale.
 - Faire défiler le temps, puis changer les valeurs du coefficient de diffusion $a$ à des fins de comparaison.


In [2]:
##############################
# Written by S.Guinchard on  #
#        05/16/23            #
##############################

################################################
######    Thermalisation de deux blocs     #####
################################################

import numpy as np
import matplotlib.pyplot as plt
size = 100        # size of the 2D grid
dx = 2. / size    # space step
dt = .001         # time step



##########################################
# Initialize a container for the profile 
# at the interface
y = np.arange(start=0, stop=size, step=1)

# Initialize a container for the temperature distribution on the grid
U = np.zeros((size, size))
V = np.zeros((size, size))



############################################
# DEF OF LAPLACIAN OPERATOR W/ FINITE DIFF #
############################################

def laplacian(Z):
    Ztop = Z[0:-2, 1:-1]
    Zleft = Z[1:-1, 0:-2]
    Zbottom = Z[2:, 1:-1]
    Zright = Z[1:-1, 2:]
    Zcenter = Z[1:-1, 1:-1]
    return (Ztop + Zleft + Zbottom + Zright -
            4 * Zcenter) / dx**2

###############################################
# UPDATE THE STATE OF THE SYSTEM TILL FINAL T #
###############################################

def diffusion(T,a):
    n = int(T / dt)   # number of iterations
    for i in range(50):
        for j in range(100):
            U[i][j]=10   #T1 value

    for i in range(50,100):
        for j in range(100):
            U[i][j]=1    #T2 value
    U0 = U
    
    fig, axes2 = plt.subplots(figsize=(12, 12))
    plt.xlabel('X', fontsize=20, fontweight='bold')
    plt.ylabel('T', fontsize=20, fontweight='bold')
    show_profile(U0,axes2,'black') # show the temperature profile @ interface and T=0.0
    
    for i in range(n):
    # We compute the Laplacian of u and v.
        deltaU = laplacian(U)

        # We take the values of u and v inside the grid.
        Uc = U[1:-1, 1:-1]
        Vc = V[1:-1, 1:-1]
        # We update the variables.
        U[1:-1, 1:-1]= \
            Uc + dt * (a * deltaU )

        # Neumann conditions: derivatives at the edges
        # are null. (ZERO GRADIENT AT THE BOUNDARY)
        for Z in (U, V):
            Z[0, :] = Z[1, :]
            Z[-1, :] = Z[-2, :]
            Z[:, 0] = Z[:, 1]
            Z[:, -1] = Z[:, -2]
    fig, ax = plt.subplots(figsize=(12, 12))
    show_TEMPERATURE(U,ax,fig)
    
    axes2.set_title(f'$T={T:.2f}$ & $a={a:.3f}$')
    show_profile(U,axes2, 'blue')
    return


#Function that displays the temperature on the grid
def show_TEMPERATURE(U, ax=None,fig=None):
    graph =  ax.imshow(U, cmap=plt.cm.copper,
              interpolation='bilinear',
              extent=[-1, 1, -1, 1])
    fig.colorbar(graph, ax=ax, label = 'T [a.u]')
    ax.set_xlabel('Y', fontsize=20, fontweight='bold')
    ax.set_ylabel('X', fontsize=20, fontweight='bold')

#Function that displays the temperature profile 
def show_profile(U, ax=None, color=None ):
    graph = ax.plot(y,U[:,int(size//2)], color=color)


interact_manual(diffusion, T=(0,100,1),a=(0.005,0.05,0.001))

interactive(children=(IntSlider(value=50, description='T'), FloatSlider(value=0.027, description='a', max=0.05…

<function __main__.diffusion(T, a)>

## 2) Diffusion bi-dimensionnelle d'une source chaude

- Ici, une source chaude à température $T_1$ diffuse sur une grille de dimension $N\times N$, le reste de l'espace étant à température $T_2<T_1$.

- La source est plus ou moins localisée, son étendue spatiale est à définir. Le lecteur peut se familiariser avec le code en changeant les paramètres, il est cependant recommandé de prendre un nombre de points de maillage de l'ordre de $10^2$ pour un temps d'execution correct. 

### Utilisation:

- Commencer par mettre le curseur temporel au minimum pour visualiser la configuration initiale. La largeur de la source est reglée par défaut sur 3 points de maillage. temp0 donne la température initiale de la source.
- Faire défiler le temps puis jouer avec les paramètres pour observer comment la chaleur diffuse sur la grille.

In [4]:
##############################
# Written by S.Guinchard on  #
#        05/16/23            #
##############################

################################################
######    Thermalisation de deux blocs     #####
################################################

import numpy as np
import matplotlib.pyplot as plt


##########################################
# Initialize a container for the profile 
# at the interface
y = np.arange(start=0, stop=size, step=1)



############################################
# DEF OF LAPLACIAN OPERATOR W/ FINITE DIFF #
############################################

def laplacian(Z):
    Ztop = Z[0:-2, 1:-1]
    Zleft = Z[1:-1, 0:-2]
    Zbottom = Z[2:, 1:-1]
    Zright = Z[1:-1, 2:]
    Zcenter = Z[1:-1, 1:-1]
    return (Ztop + Zleft + Zbottom + Zright -
            4 * Zcenter) / dx**2

###############################################
# UPDATE THE STATE OF THE SYSTEM TILL FINAL T #
###############################################

def diffusion(T,a,width, temp0):
    size = 100        # size of the 2D grid
    U = np.zeros((size, size))
    V = np.zeros((size, size))
    dx = 2. / size    # space step
    dt = .0005         # time step
    n = int(T / dt)   # number of iterations
    length = float(size)
    for i in range (int(length/2)-width,int(length/2)+width):
        for j in range (int(length/2)-width,int(length/2)+width):
            U[i][j]=temp0

    U0 = U
    
    fig, axes2 = plt.subplots(figsize=(12, 12))
    plt.xlabel('X', fontsize=20, fontweight='bold')
    plt.ylabel('T', fontsize=20, fontweight='bold')
    show_profile(U0,axes2,'black') # show the temperature profile @ interface and T=0.0
    
    for i in range(n):
    # We compute the Laplacian of u and v.
        deltaU = laplacian(U)

        # We take the values of u and v inside the grid.
        Uc = U[1:-1, 1:-1]
        Vc = V[1:-1, 1:-1]
        # We update the variables.
        U[1:-1, 1:-1]= \
            Uc + dt * (a*1e-4 * deltaU )

        # Neumann conditions: derivatives at the edges
        # are null. (ZERO GRADIENT AT THE BOUNDARY)
        for Z in (U, V):
            Z[0, :] = Z[1, :]
            Z[-1, :] = Z[-2, :]
            Z[:, 0] = Z[:, 1]
            Z[:, -1] = Z[:, -2]
    fig, ax = plt.subplots(figsize=(12, 12))
    show_TEMPERATURE(U,ax,fig)
    
    axes2.set_title(f'$T={T:.2f}$ & $a={a:.3f}e-4$')
    show_profile(U,axes2, 'blue')
    return

#Function that displays the temperature on the grid
def show_TEMPERATURE(U, ax=None,fig=None):
    graph =  ax.imshow(U, cmap=plt.cm.copper,
              interpolation='bilinear',
              extent=[-1, 1, -1, 1])
    fig.colorbar(graph, ax=ax, label = 'T [a.u]')
    ax.set_xlabel('Y', fontsize=20, fontweight='bold')
    ax.set_ylabel('X', fontsize=20, fontweight='bold')

#Function that displays the temperature profile 
def show_profile(U, ax=None, color=None ):
    graph = ax.plot(y,U[:,int(size//2)], color=color)


interact_manual(diffusion,T=(0,100,1),a=(1,10,1),width=(1,5,1),temp0=(1,20,1))

interactive(children=(IntSlider(value=50, description='T'), IntSlider(value=5, description='a', max=10, min=1)…

<function __main__.diffusion(T, a, width, temp0)>